### Carga de paquetes

In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf
import os
import glob
import cv2
import matplotlib.pyplot as plt
from Funciones import leer_fotos_y_guardar_por_lotes

In [2]:
root_path = "../"

directorio_origen = root_path + "Mas_caras"

destino = root_path + "Caras_cortadas" #carpeta donde se guardaran las imagenes de caras cortadas

Convertimos los archivos formatio hiec a jpg

Funcion que lee las fotos y si encuentra fotos formato hiec las convierte a jpg

In [3]:
directorio_destino="../lotes"
fotos=leer_fotos_y_guardar_por_lotes(directorio_origen,directorio_destino)

In [4]:
import os
import pickle
import glob

def mostrar_informacion_lotes(directorio_lotes):
    """
    Muestra información sobre las personas y la cantidad de fotos en los lotes guardados en el directorio.

    Args:
        directorio_lotes (str): Ruta del directorio que contiene los lotes guardados en formato .pkl.
    """
    nombres_personas = {}
    
    # Recorrer todos los archivos de lotes
    for archivo in glob.glob(os.path.join(directorio_lotes, "*.pkl")):
        # Obtener el nombre de la persona desde el archivo
        nombre_persona = os.path.splitext(os.path.basename(archivo))[0].split("_lote")[0]
        
        # Cargar el archivo de lote
        with open(archivo, "rb") as f:
            lote_imagenes = pickle.load(f)

        # Agregar la cantidad de imágenes al conteo de la persona
        if nombre_persona in nombres_personas:
            nombres_personas[nombre_persona] += len(lote_imagenes)
        else:
            nombres_personas[nombre_persona] = len(lote_imagenes)

    # Mostrar la información
    print("Cantidad de personas:", len(nombres_personas))
    print("Cantidad de fotos por persona:")
    for nombre_persona, cantidad_fotos in nombres_personas.items():
        print(f"- {nombre_persona}: {cantidad_fotos}")


In [5]:
# Mostrar información
nombres_personas = list(fotos.keys())
print("Cantidad de personas:", len(nombres_personas))
print("Cantidad de fotos por persona:")
for nombre_persona, fotos_persona in fotos.items():
    print(f"- {nombre_persona}: {len(fotos_persona)}")



AttributeError: 'NoneType' object has no attribute 'keys'

## funcion para imprimir fotos de una persona

In [ ]:

#from Funciones import show_people
#show_people(fotos)

## Funcion para imprimir X cantidad de fotos por persona

In [ ]:
#from Funciones import show_people_agg_cant
#show_people_agg_cant(fotos,6)

## Funcion para recortar imagenes

## Recortar imagen

In [6]:
import os
import cv2
import matplotlib.pyplot as plt

def recortar_imagen(image):
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Utilizar un clasificador específico para caras
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

    # Detectar rostros en la imagen
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))

    # Recorrer las caras detectadas
    face_images = []
    for (x, y, w, h) in faces:
        # Recortar la cara de la imagen
        face_images.append(image[y:y+h, x:x+w])

    return face_images

## Procesar Carpetas

In [7]:
import os
import pickle
import cv2

def procesar_lotes(directorio_lotes, directorio_destino):
    """
    Procesa imágenes guardadas por lotes, recorta y guarda en escala de grises.

    Args:
        directorio_lotes (str): Ruta del directorio donde están los lotes guardados.
        directorio_destino (str): Ruta del directorio donde se guardarán las imágenes procesadas.
    """
    # Crear el directorio de destino si no existe
    if not os.path.exists(directorio_destino):
        os.makedirs(directorio_destino)

    # Inicializar un diccionario para llevar el conteo por persona
    contadores_por_persona = {}

    # Iterar sobre los archivos de lotes
    for root, dirs, files in os.walk(directorio_lotes):
        for file_name in files:
            if file_name.endswith(".pkl"):
                lote_path = os.path.join(root, file_name)
                print(f"Procesando lote: {lote_path}")

                # Cargar el lote
                with open(lote_path, "rb") as f:
                    lote_imagenes = pickle.load(f)

                # Determinar nombre de persona desde el archivo del lote
                nombre_persona = os.path.splitext(file_name)[0].split("_lote")[0]

                # Crear carpeta de destino para la persona si no existe
                dir_path_recorte = os.path.join(directorio_destino, nombre_persona)
                if not os.path.exists(dir_path_recorte):
                    os.makedirs(dir_path_recorte)

                # Inicializar contador global para la persona si no existe
                if nombre_persona not in contadores_por_persona:
                    contadores_por_persona[nombre_persona] = 1

                # Obtener el contador actual
                contador_imagenes = contadores_por_persona[nombre_persona]

                for image_buffer in lote_imagenes:
                    if image_buffer is not None:
                        # Decodificar el buffer de bytes de la imagen
                        image = cv2.imdecode(image_buffer, cv2.IMREAD_COLOR)

                        if image is not None:
                            # Recortar la imagen
                            face_images = recortar_imagen(image)

                            for face_image in face_images:
                                # Convertir la imagen a escala de grises
                                gray_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)

                                # Generar el nombre del archivo
                                output_path = os.path.join(
                                    dir_path_recorte,
                                    f"{nombre_persona}_{contador_imagenes}.jpg"
                                )

                                # Guardar la imagen recortada en escala de grises
                                cv2.imwrite(output_path, gray_image)
                                print(f"Imagen guardada: {output_path}")

                                # Incrementar el contador
                                contador_imagenes += 1

                # Actualizar el contador global para la persona
                contadores_por_persona[nombre_persona] = contador_imagenes


In [8]:
procesar_lotes(directorio_destino, destino)

Procesando lote: ../lotes\Abel_lote_0.pkl
Imagen guardada: ../Caras_cortadas\Abel\Abel_1.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_2.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_3.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_4.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_5.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_6.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_7.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_8.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_9.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_10.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_11.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_12.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_13.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_14.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_15.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_16.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_17.jpg
Imagen guardada: ../Caras_cortadas\Abel\Abel_18.jpg
Imagen guardada: ../Caras_corta

In [ ]:
from Funciones import  Leer_fotos
fotos_c=Leer_fotos(destino)

In [ ]:
# Mostrar información
nombres_personas = list(fotos_c.keys())
print("Cantidad de personas:", len(nombres_personas))
print("Cantidad de fotos por persona:")
for nombre_persona, fotos_persona in fotos_c.items():
    
    print(f"- {nombre_persona}: {len(fotos_persona)}")